# 1) OBTENER LA SERIE DE PRECIOS PARA ACTIVOS EN DIVERSAS TEMPORALIDADES

### Exchange: BingX


In [9]:
from backtesting.fechas import past_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio
from concurrent import futures
import requests, datetime, pickle
import locale, os
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr
import pandas as pd


locale.setlocale(locale.LC_ALL, 'es_ES');

In [10]:
# SE UTILIZA EN EL THREAD POOL
def apply_requests(args):
    service, params = args
    response = requests.get(service, params=params)
    return response

# SE UTILIZA EN THREAD POOL
def apply_squeeze_momentum_indicator(ARGS):
    data, parametros = ARGS
    activos = list(data.keys())
    temporalidades = list(data[activos[0]].keys())

    SMI = {}
    for activo in activos:
        SMI[activo] = {}
        for t in temporalidades:
            df = data[activo][t].iloc[:]
            df = squeeze_momentum_indicator(df, **parametros)
            SMI[activo][t] = df.dropna()
    return SMI

# DECORADOR
def styler(func):
    def wrapper(*args, **kwargs):
        df = func(*args, **kwargs)
        # formato para la impresion con estilo
        keys= list(df.columns)
        values = ['{:.2f}' for i in range(len(keys))]
        format_dict = dict(zip(keys, values))
        return df.style.format(format_dict).background_gradient(cmap='PRGn')
    return wrapper

# SE UTILIZA PARA VISUALIZAR LOS DATOS DEL OBJETO correlaciones
@styler
def get_correlation(data, activo):
    # Filter the data dictionary based on the activo
    filtered_data = {
        key: values[activo.upper()]
        for key, values in data.items()
    }
    # Create the DataFrame
    df = pd.DataFrame(filtered_data)
    df.columns.name = 'correlacion {}'.format(activo)
    return df


In [14]:
## SELECCION: ACTIVOS PARA DESCARGAR
activos = ['BTC', 'LTC', 'XRP', 'NEO']
activos = ['BTC', 'XRP', 'APE']
activos = ['ETH', 'LINK', 'BNB', 'NEO']

## SELECCION: TEMPORALIDADES PARA DESCARGAR
data_long_term = True
data_intermediate_term = True
data_short_term = False

## VARIABLES DE ENTORNO
URL = 'https://open-api.bingx.com'
path = '/openApi/swap/v2/quote/klines'
service = URL + path
limit = 1000
now = datetime.datetime.now()
endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
fecha = now.strftime('%d_%B_%Y')

## CREACION DE DIRECTORIO
dir_path = os.path.join('data', fecha)
if crear_directorio(dir_path):
    contador = '1'
else:
    # obtener una lista con los ficheros de dir_path
    ficheros = os.listdir(dir_path)
    # con list comprehension hacer split a cada lista y quedarse con el primer elemento
    lista_indices = [int(f.split('_')[0]) for f in ficheros]
    if len(lista_indices) == 0:
        contador = '1'
    else:
        # ordenar la lista y crear el contador con el ultimo elemento + 1
        lista_indices.sort()
        contador = str(lista_indices[-1] + 1)




## PROGRAMA PRINCIPAL
## 1) SELECCION DE LA VENTANA DE TIEMPO A DESCARGAR
temporalidades = []
start_time = []
if data_long_term:
    temporalidades.append('1d')
    start_time.append(int(past_timestamp(555, 'days', now)))
if data_intermediate_term:
    temporalidades.append('4h')
    start_time.append(int(past_timestamp(76, 'days', now)))
    temporalidades.append('2h')
    start_time.append(int(past_timestamp(50, 'days', now)))
    temporalidades.append('1h')
    start_time.append(int(past_timestamp(25, 'days', now)))
if data_short_term:
    temporalidades.append('30m')
    start_time.append(int(past_timestamp(14, 'days', now)))
    temporalidades.append('15m')
    start_time.append(int(past_timestamp(7, 'days', now)))
    temporalidades.append('5m')
    start_time.append(int(past_timestamp(2, 'days', now)))
    temporalidades.append('3m')
    start_time.append(int(past_timestamp(32, 'hour', now)))
    temporalidades.append('1m')
    start_time.append(int(past_timestamp(12, 'hour', now)))

## 2) ESTRUCTURACION DE LOS PARAMETROS
ARGS = []
for activo in activos:
    symbol = activo+'-USDT'
    for i in range(len(temporalidades)):
        ARGS.append(
            (
                service, 
                {
                    'symbol':symbol, 
                    'interval':temporalidades[i],
                    'limit':limit,
                    'startTime':start_time[i],
                    'endTime':endTime
                }
            )
        )

## 3) THREAD POOL: requests.get
with futures.ThreadPoolExecutor(max_workers=None) as pool:
    response = list(pool.map(apply_requests, ARGS))

## 4) ESTRUCTURACION Y AJUSTE DE LOS DATOS
df = [r.json() for r in response]
df = [response_to_dataframe(r) for r in df]

for i in range(len(df)):
    df[i]['time'] = df[i]['time'].apply(retrasar_tres_horas)
    df[i] = df[i].set_index('time')

# 5) CREANDO OBJETO DE TRABAJO Y EXPORTACION
assets_dict = {}
for i, activo in enumerate(activos):
    assets_dict[activo] = {}
    for j, t in enumerate(temporalidades):
        assets_dict[activo][t] = df[i*len(temporalidades)+j]

# 6) EXPORTACION DE data_raw
file_name = f'{contador}_' + '_'.join(activos+temporalidades)
file_dir = os.path.join(dir_path, file_name)
with open(f'{file_dir}.pickle', 'wb') as f:
    pickle.dump(assets_dict, f, pickle.HIGHEST_PROTOCOL)


# 7) GENERANDO MULTIPLES PARAMETROS DE ENTRADA
data = assets_dict
parametros = {'length': 20, 'mult': 2, 'length_KC': 20, 'mult_KC': 1.5, 'length_atr': 10, 'use_EMA': True}
length_options = [9,12,14,18,20,22,30,40,50,60,70,80,90]

lista_diccionarios = []
for length in length_options:
    nuevo_parametro = parametros.copy()
    nuevo_parametro['length'] = length
    nuevo_parametro['length_KC'] = length
    lista_diccionarios.append(nuevo_parametro)

ARGS = [(data, d) for d in lista_diccionarios]

# 8) THREAD POOL: squeeze_momentum_indicator
data_SQ = {}
with futures.ThreadPoolExecutor(max_workers=None) as pool:
    results = list(pool.map(apply_squeeze_momentum_indicator, ARGS))
    for i, length in enumerate(length_options):
        data_SQ[length] = results[i]

# 9) EXPORTACION DE data_SQ
file_name = f'{contador}_' + '_'.join(activos+temporalidades)
file_name += '_SQ'
file_dir = os.path.join(dir_path, file_name)
with open(f'{file_dir}.pickle', 'wb') as f:
    pickle.dump(data_SQ, f, pickle.HIGHEST_PROTOCOL)

In [15]:
# PARTE 2
# 10) CREACION DEL OBJETO time_window

now = datetime.datetime.now()
time_window = {
    '1d': {50: now-datetime.timedelta(days=286), 40: now-datetime.timedelta(days=286),
    30: now-datetime.timedelta(days=120), 22: now-datetime.timedelta(days=120),
    20: now-datetime.timedelta(days=120), 18: now-datetime.timedelta(days=120),
    14: now-datetime.timedelta(days=77), 12: now-datetime.timedelta(days=40),
    9: now-datetime.timedelta(days=40)},
    '4h': {50: now-datetime.timedelta(days=50), 40: now-datetime.timedelta(days=50),
    30: now-datetime.timedelta(days=25), 22: now-datetime.timedelta(days=25),
    20: now-datetime.timedelta(days=25), 18: now-datetime.timedelta(days=25),
    14: now-datetime.timedelta(days=12), 12: now-datetime.timedelta(days=7),
    9: now-datetime.timedelta(days=7)},
    '2h': {50: now-datetime.timedelta(days=22), 40: now-datetime.timedelta(days=22),
    30: now-datetime.timedelta(days=14), 22: now-datetime.timedelta(days=14),
    20: now-datetime.timedelta(days=14), 18: now-datetime.timedelta(days=14),
    14: now-datetime.timedelta(days=7), 12: now-datetime.timedelta(days=4),
    9: now-datetime.timedelta(days=4)},
    '1h': {50: now-datetime.timedelta(days=9), 40: now-datetime.timedelta(days=9),
    30: now-datetime.timedelta(days=4), 22: now-datetime.timedelta(days=4),
    20: now-datetime.timedelta(days=4), 18: now-datetime.timedelta(days=4),
    14: now-datetime.timedelta(days=3), 12: now-datetime.timedelta(days=1),
    9: now-datetime.timedelta(days=1)},
    '30m': {50: now-datetime.timedelta(days=4), 40: now-datetime.timedelta(days=4),
    30: now-datetime.timedelta(days=2), 22: now-datetime.timedelta(days=2),
    20: now-datetime.timedelta(days=2), 18: now-datetime.timedelta(days=2),
    14: now-datetime.timedelta(days=1), 12: now-datetime.timedelta(hours=13),
    9: now-datetime.timedelta(hours=13)},
    '15m': {50: now-datetime.timedelta(days=2), 40: now-datetime.timedelta(days=2),
    30: now-datetime.timedelta(days=1), 22: now-datetime.timedelta(days=1),
    20: now-datetime.timedelta(days=1), 18: now-datetime.timedelta(days=1),
    14: now-datetime.timedelta(hours=13), 12: now-datetime.timedelta(hours=10),
    9: now-datetime.timedelta(hours=10)},
    '5m': {50: now-datetime.timedelta(days=1), 40: now-datetime.timedelta(days=1),
    30: now-datetime.timedelta(hours=13), 22: now-datetime.timedelta(hours=13),
    20: now-datetime.timedelta(hours=13), 18: now-datetime.timedelta(hours=13),
    14: now-datetime.timedelta(hours=7), 12: now-datetime.timedelta(hours=5),
    9: now-datetime.timedelta(hours=5)},
    '3m': {50: now-datetime.timedelta(hours=12), 40: now-datetime.timedelta(hours=12),
    30: now-datetime.timedelta(hours=7), 22: now-datetime.timedelta(hours=7),
    20: now-datetime.timedelta(hours=7), 18: now-datetime.timedelta(hours=7),
    14: now-datetime.timedelta(hours=4), 12: now-datetime.timedelta(hours=3),
    9: now-datetime.timedelta(hours=3)},
    '1m': {50: now-datetime.timedelta(hours=4), 40: now-datetime.timedelta(hours=4),
    30: now-datetime.timedelta(hours=3), 22: now-datetime.timedelta(hours=3),
    20: now-datetime.timedelta(hours=3), 18: now-datetime.timedelta(hours=3),
    14: now-datetime.timedelta(hours=2), 12: now-datetime.timedelta(hours=1),
    9: now-datetime.timedelta(hours=1)}
    }


time_window = {
    interval: {key: value.strftime("%Y-%m-%d %H:%M:%S") for key, value in values.items()}
    for interval, values in time_window.items()
}


# 11) EXPORTACION DE time_window
file_name = f'{contador}_' + '_'.join(activos+temporalidades)
file_name += '_TW'
file_dir = os.path.join(dir_path, file_name)
with open(f'{file_dir}.pickle', 'wb') as f:
    pickle.dump(time_window, f, pickle.HIGHEST_PROTOCOL)

In [16]:
# PARTE 3
# 12) OBTENIENDO LAS COLAS DE LOS DATOS SEGUN time_windows
# FILTRANDO LOS PERIODOS PARA CADA WINDOW
filtered_dict = {}
for param in data_SQ:
    filtered_dict[param] = {}
    for activo in data_SQ[param]:
        filtered_dict[param][activo] = {}
        for temporalidad in data_SQ[param][activo]:
            if param not in time_window[temporalidad]:
                filtered_dict[param][activo][temporalidad] = data_SQ[param][activo][temporalidad]
            else:
                filtered_dict[param][activo][temporalidad] = data_SQ[param][activo][temporalidad].loc[time_window[temporalidad][param]:]


# 13) OBTENIENDO EL OBJETO CORRELACION CON LOS VALORES DE CORRELACION
correlacion = {}
for param in filtered_dict:

    correlacion[param] = {}

    for activo in filtered_dict[param]:
        correlacion[param][activo] = {}

        for temporalidad in filtered_dict[param][activo]:
            df = filtered_dict[param][activo][temporalidad]
            correlacion[param][activo][temporalidad] = {}
            try:
                correlacion[param][activo][temporalidad], _ = pearsonr(df['value'], df['close'])
                correlacion[param][activo][temporalidad] = round(correlacion[param][activo][temporalidad], 2)
            except:
                print(f"Temporalidad vacia {param} {activo} {temporalidad}")
                correlacion[param][activo][temporalidad] = -10

# 14) EXPORTACION DE correlacion 
file_name = f'{contador}_' + '_'.join(activos+temporalidades)
file_name += '_CORR'
file_dir = os.path.join(dir_path, file_name)
with open(f'{file_dir}.pickle', 'wb') as f:
    pickle.dump(correlacion, f, pickle.HIGHEST_PROTOCOL)


In [18]:
if __name__ == '__main__':
    activo = 'eth'
    df = get_correlation(correlacion, activo)
    display(df)

correlacion eth,9,12,14,18,20,22,30,40,50,60,70,80,90
1d,0.60,0.65,0.35,0.26,0.25,0.25,0.27,0.43,0.49,0.41,0.42,0.43,0.47
4h,0.69,0.62,0.58,0.38,0.37,0.37,0.33,0.20,0.22,0.57,0.49,0.16,0.03
2h,0.53,0.71,0.68,0.36,0.35,0.35,0.41,0.45,0.45,0.45,0.42,0.44,0.49
1h,0.70,0.79,0.59,0.54,0.60,0.66,0.75,0.60,0.54,0.42,0.42,0.44,0.47


# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [ ]:
from backtesting.indicadores import backtest_entry
import plotly.graph_objects as go

In [ ]:
entradas_backtest = {}
for activo in temporalidades_filtradas:
    for t in temporalidades_filtradas[activo]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activo][t])
        dict_key = activo + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [ ]:
## Obtenemos los dataframes de los activos y temporalidades filtradas, con indicador aplicado y parámetros default
## Visualizar las entradas generadas

ultimas_velas = 100

for activo in SMI_default:
    if len(temporalidades_filtradas[activo]) > 0:
        for t in temporalidades_filtradas[activo]:
            # print(f'{activo} {t}')
            # display(SMI_default[activo][t])
            # print()
            df = SMI_default[activo][t].iloc[-ultimas_velas:]

            candlestick = go.Candlestick(x=df.index,
                            open=df['open'],
                            high=df['high'],
                            low=df['low'],
                            close=df['close'])

            upper_band = go.Scatter(x=df.index, y=df['upper_BB'], name='upper_band', line=dict(color='red', width=1))
            lower_band = go.Scatter(x=df.index, y=df['lower_BB'], name='lower_band', line=dict(color='red', width=1))
            
            upper_KC = go.Scatter(x=df.index, y=df['upper_KC'], name='upper_KC', line=dict(color='blue', width=1, dash='dash'))
            lower_KC = go.Scatter(x=df.index, y=df['lower_KC'], name='lower_KC', line=dict(color='blue', width=1, dash='dash'))
            
            bar = go.Bar(x=df.index, y=df['value'], name='squeeze')


            fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_KC, lower_KC])
            fig2 = go.Figure(data=[bar])
            
            fig.layout.xaxis.type = 'category'
            fig.layout.xaxis.rangeslider.visible = False
            # update figure 1
            fig.update_layout(
                autosize=False,
                width=1500,
                height=600,
                margin=dict(
                    
                    l=50,
                    r=50,
                    b=100,
                    t=50,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
                title=f'{activo} {t}'
            )
            # update figure 2
            fig2.update_layout(
                autosize=False,
                width=1500,
                height=300,
                margin=dict(
                    
                    l=50,
                    r=150,
                    b=1,
                    t=10,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
            )

            for k in entradas_backtest:
                if k.startswith(f'{activo}_{t}') and k.endswith('short'):
                    for x_value in entradas_backtest[f'{activo}_{t}_short'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )

                if k.startswith(f'{activo}_{t}') and k.endswith('long'):
                    for x_value in entradas_backtest[f'{activo}_{t}_long'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )

# agregar unas lineas horizontales a un ATR de distancia


            fig.show()
            fig2.show()            

# Optimizar los parámetros del SMI para la temporalidad DIARIA y verificar la correlacion en las otras temporalidades

SMI_optimized = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame()},<br />
    'ETH': <br />
    {'1d': pd.DataFrame()},<br />
    'XRP':<br />
    {'1d': pd.DataFrame()}<br />
<br />
}<br />

In [ ]:
import numpy as np
from tqdm import tqdm
from copy import copy

# import warnings
# warnings.filterwarnings("ignore")

import sys
sys.exit(0)

SystemExit: 0

c:\Users\Agustin\Desktop\Zartex\BingX-Backtesting\venv\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
#TODO: threads
SMI_optimized = {}
SMI_optimized_corr = {}

length_range = range(18, 23)
mult_range = np.arange(1.5, 2.6, 0.1)
length_KC_range = range(18, 23)
mult_KC_range = np.arange(1, 2.1, 0.1)


# activo = 'BTC'
for activo in assets_dict:
    best_corr = -1

    best_params = None


    for length in length_range:
        for mult in mult_range:
            for length_KC in length_KC_range:
                for mult_KC in tqdm(mult_KC_range):
                    df = assets_dict[activo]['1h'].copy()
                    # Aplicar la función squeeze_momentum_indicator con los parámetros actuales
                    params = (length, mult, length_KC, mult_KC)
                    df = squeeze_momentum_indicator(df, params[0], params[1], params[2], params[3], length_atr=10, use_EMA=True)
                    df = df.dropna()
                    try:
                        corr, _ = pearsonr(df['value'], df['close'])
                        # Actualizar los mejores parámetros y coeficiente de correlación si se encuentra una mejora
                        if corr > best_corr:
                            best_corr = corr
                            best_params = params
                            
                            SMI_optimized.update({activo: df})
                            SMI_optimized_corr.update({f'{activo}_best_corr': best_corr, f'{activo}_best_params': best_params})
                    
                    except:
                        continue
                        


100%|██████████| 11/11 [00:00<00:00, 22.03it/s]


In [ ]:
SMI_optimized_corr

{'NEO_best_corr': 0.194909269201475, 'NEO_best_params': (22, 1.5, 18, 1.0)}

In [ ]:
assets_dict['BTC']['1d']

,open,close,high,low,volume
time,,,,,
2022-05-13 21:00:00,29296.3,30092.7,30330.5,28644.9,20285.23
2022-05-14 21:00:00,30081.4,31328.5,31444.7,29496.4,21410.99
2022-05-15 21:00:00,31331.0,29868.8,31331.7,29100.3,38087.72
2022-05-16 21:00:00,29868.6,30450.6,30783.6,29455.8,36384.99
2022-05-17 21:00:00,30443.9,28714.2,30699.1,28661.1,40287.31
...,...,...,...,...,...
2023-05-07 21:00:00,28422.5,27658.0,28623.0,27256.5,73696.65
2023-05-08 21:00:00,27658.0,27618.9,27805.0,27344.1,41152.73
2023-05-09 21:00:00,27618.9,27596.4,28307.6,26782.8,75589.71


In [ ]:
print('TEMPORALIDAD DIARIA')
print('Correlación parámetros por default')
for activo in SMI_default_corr:
    for t in SMI_default_corr[activo]:
        if t == '1d':
            print(f'{activo} 1d corr: {round(SMI_default_corr[activo][t], 2)}')

print()
print('Correlación parámetros optimizados')
for corr in SMI_optimized_corr:
    if corr.endswith('best_corr'):
        print(f'{corr[:-10]} 1d corr: {round(SMI_optimized_corr[corr], 2)}')


TEMPORALIDAD DIARIA
Correlación parámetros por default
BTC 1d corr: 0.36
ETH 1d corr: 0.4
XRP 1d corr: 0.4
ENS 1d corr: 0.44

Correlación parámetros optimizados
BTC 1d corr: 0.4
ETH 1d corr: 0.41
XRP 1d corr: 0.42
ENS 1d corr: 0.46


# Verificar como varía la correlación en las otras temporalidades

In [ ]:
SMI_default_optimized = {}
SMI_default_optimized_corr = {}

for activo in assets_dict:
    SMI_default_optimized[activo] = {}
    SMI_default_optimized_corr[activo] = {}
    for t in temporalidades:
        df = assets_dict[activo][t].iloc[:]
        length, mult, length_KC, mult_KC = SMI_optimized_corr[f'{activo}_best_params']
        df = squeeze_momentum_indicator(df, length, mult, length_KC, mult_KC, length_atr=100, use_EMA=False)
        df = df.dropna()
        SMI_default_optimized[activo][t] = df
        df = df[['value', 'close']]
        corr, _ = pearsonr(df['value'], df['close'])
        SMI_default_optimized_corr[activo][t] = corr
        

print('Correlaciones')
for activo in assets_dict:
    for t in temporalidades:
        print(f'{activo} {t} ->  old: {round(SMI_default_corr[activo][t], 2)} | new: {round(SMI_default_optimized_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d ->  old: 0.36 | new: 0.42
BTC 4h ->  old: 0.57 | new: 0.52
BTC 2h ->  old: 0.58 | new: 0.6
BTC 1h ->  old: 0.42 | new: 0.48
BTC 30m ->  old: 0.07 | new: 0.47
BTC 15m ->  old: 0.27 | new: 0.09

ETH 1d ->  old: 0.4 | new: 0.34
ETH 4h ->  old: 0.48 | new: 0.44
ETH 2h ->  old: 0.58 | new: 0.55
ETH 1h ->  old: 0.49 | new: 0.44
ETH 30m ->  old: 0.05 | new: 0.27
ETH 15m ->  old: 0.25 | new: 0.13

XRP 1d ->  old: 0.4 | new: 0.47
XRP 4h ->  old: 0.32 | new: 0.29
XRP 2h ->  old: 0.41 | new: 0.21
XRP 1h ->  old: 0.25 | new: 0.22
XRP 30m ->  old: -0.06 | new: 0.54
XRP 15m ->  old: 0.48 | new: 0.38

ENS 1d ->  old: 0.44 | new: 0.56
ENS 4h ->  old: 0.39 | new: 0.34
ENS 2h ->  old: 0.42 | new: 0.28
ENS 1h ->  old: 0.28 | new: 0.24
ENS 30m ->  old: 0.04 | new: 0.4
ENS 15m ->  old: 0.4 | new: 0.16



In [ ]:
assets_dict['BTC']['4h']

,open,close,high,low,volume
time,,,,,
2022-05-31 21:00:00,31795.9,29796.7,31983.5,29330.8,88397.05
2022-06-01 21:00:00,29803.4,30454.3,30687.7,29596.4,61620.08
2022-06-02 21:00:00,30454.3,29698.5,30692.1,29310.4,45464.50
2022-06-03 21:00:00,29702.5,29869.8,29970.0,29492.1,18033.88
2022-06-04 21:00:00,29869.9,29918.8,30182.3,29536.7,17346.73
...,...,...,...,...,...
2023-05-25 21:00:00,26462.4,26703.6,26936.5,26317.2,34495.05
2023-05-26 21:00:00,26703.3,26852.9,26889.0,26534.7,14347.49
2023-05-27 21:00:00,26852.9,28073.9,28266.9,26765.4,42694.99


# Aplicar las condiciones de la estrategia a las temporalidades que ya se habían aplicado

In [ ]:
## AJUSTE MANUAL DE length_atr y use_EMA para BTC
var1 = assets_dict['BTC']['1h'].iloc[:]
params = SMI_optimized_corr['BTC_best_params']
print(params)
var2 = squeeze_momentum_indicator(var1,22, params[1], 22,params[3],100,True)
var2 = var2.dropna()
var3, _ = pearsonr(var2['value'], var2['close'])
round(var3, 3)


(22, 1.5, 22, 1.0)


0.446

In [ ]:
var2

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-04-24 16:00:00,27327.0,27370.5,27404.0,27310.0,975.72,27780.443793,27202.056207,94.0,27681.766833,27300.733167,-244.813196,False
2023-04-24 17:00:00,27370.5,27448.5,27470.0,27328.0,1009.96,27773.096449,27197.267187,142.0,27674.737922,27295.625715,-227.626707,False
2023-04-24 18:00:00,27448.5,27403.5,27448.5,27313.5,829.90,27764.491541,27189.917550,135.0,27665.680330,27288.728761,-217.036741,False
2023-04-24 19:00:00,27403.5,27453.5,27503.0,27370.5,980.07,27729.401679,27193.916503,132.5,27649.026444,27274.291738,-178.496766,False
2023-04-24 20:00:00,27453.5,27499.0,27547.0,27433.5,1130.77,27702.720175,27198.552552,113.5,27636.540997,27264.731730,-129.696011,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 16:00:00,26290.3,26453.4,26475.3,25810.2,9988.60,26847.612802,26117.823562,665.1,26686.724013,26278.712351,-280.856854,False
2023-05-12 17:00:00,26453.4,26430.4,26481.8,26355.0,3043.40,26800.264065,26122.672299,126.8,26663.945184,26258.991179,-218.380057,False
2023-05-12 18:00:00,26430.4,26742.5,26804.0,26430.4,4615.50,26761.846833,26140.662258,373.6,26657.120122,26245.388969,-97.834926,False


In [ ]:

entrada_short, entrada_long = backtest_entry(var2)
        
ultimas_velas = 400


var2 = SMI_default[activo][t].iloc[-ultimas_velas:]

candlestick = go.Candlestick(x=var2.index,
                open=var2['open'],
                high=var2['high'],
                low=var2['low'],
                close=var2['close'])

upper_band = go.Scatter(x=var2.index, y=var2['upper_BB'], name='upper_band', line=dict(color='red', width=1))
lower_band = go.Scatter(x=var2.index, y=var2['lower_BB'], name='lower_band', line=dict(color='red', width=1))

upper_KC = go.Scatter(x=var2.index, y=var2['upper_KC'], name='upper_KC', line=dict(color='blue', width=1, dash='dash'))
lower_KC = go.Scatter(x=var2.index, y=var2['lower_KC'], name='lower_KC', line=dict(color='blue', width=1, dash='dash'))

bar = go.Bar(x=var2.index, y=var2['value'], name='squeeze')


fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_KC, lower_KC])
fig2 = go.Figure(data=[bar])

fig.layout.xaxis.type = 'category'
fig.layout.xaxis.rangeslider.visible = False
# update figure 1
fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    margin=dict(
        
        l=50,
        r=50,
        b=100,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
    # title=f'{activo} {t}'
)
# update figure 2
fig2.update_layout(
    autosize=False,
    width=1500,
    height=300,
    margin=dict(
        
        l=50,
        r=150,
        b=1,
        t=10,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

if len(entrada_short) > 0:
    for x_value in entrada_short.index:
        if x_value in var2.index:
            fig.add_shape(
                type="line",
                x0=x_value,
                y0=var2['low'].min(),
                x1=x_value,
                y1=var2['high'].max(),
                line=dict(color="red", width=1, dash="dash"),
            )
            fig2.add_shape(
                type="line",
                x0=x_value,
                y0=var2['value'].min(),
                x1=x_value,
                y1=var2['value'].max(),
                line=dict(color="red", width=1, dash="dash"),
            )

if len(entrada_long) > 0:
    for x_value in entrada_long.index:
        if x_value in var2.index:
            fig.add_shape(
                type="line",
                x0=x_value,
                y0=var2['low'].min(),
                x1=x_value,
                y1=var2['high'].max(),
                line=dict(color="green", width=1, dash="dash"),
            )
            fig2.add_shape(
                type="line",
                x0=x_value,
                y0=var2['value'].min(),
                x1=x_value,
                y1=var2['value'].max(),
                line=dict(color="green", width=1, dash="dash"),
            )

fig.show()
fig2.show()            

In [ ]:
entrada_short

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,


In [ ]:
## TODO: Esto de acá fue una prueba y no es relevante para la función del módulo actual
import sys
sys.exit(0)

for señales in entradas_backtest:
    print(señales)
    display(entradas_backtest[señales].iloc[-1].to_frame().T)
    print()

SystemExit: 0

# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales